In [1]:
import json
import pandas as pd
from rdflib.namespace import DC, FOAF
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef, Namespace

### Input data

In [2]:
input_file = "/home/chris/Workspace/app/anacDATA/lil_anacDataset_postprocessed.tsv"
df = pd.read_csv(input_file, sep="\t", dtype=object)
# df.head()
df.shape

(100, 17)

### Prefixes

In [3]:
g = Graph()

tds = Namespace("http://td.com/schema/")
g.bind("tds", URIRef("http://td.com/schema/"))

rel = Namespace("http://td.com/schema/rel/")
g.bind("rel", URIRef("http://td.com/schema/rel/"))

pa = Namespace("http://td.com/pa/")
g.bind("pa", URIRef("http://td.com/pa/"))

gara = Namespace("http://td.com/gara/")
g.bind("gara", URIRef("http://td.com/gara/"))

fornitore = Namespace("http://td.com/fornitore/")
g.bind("fornitore", URIRef("http://td.com/fornitore/"))

g.bind("rdf", URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#"))

### tsv to triples

In [4]:
num_gara = 0
num_pa = 0
num_forni = 0

for index, row in df.iterrows():
    curr_gara = URIRef(gara + "g" + str(num_gara))
    g.add( (curr_gara, RDF.type, tds.gara) )
    
    # --
    if row['cig'] != "0000000000":
        g.add( (curr_gara, rel.cig, Literal(row['cig'])) )
    
    # -- 
    curr_pa = g.value(None, rel.cf, Literal(row['cfStrutturaProponente']))
    if not curr_pa:
        curr_pa = URIRef(pa + "p" + str(num_pa))
        num_pa += 1
    g.add( (curr_pa, rel.cf, Literal(row['cfStrutturaProponente'])) )
    g.add( (curr_pa, rel.denominazione, Literal(row['denominazioneStrutturaProponente'])) )
    g.add( (curr_pa, RDF.type, tds.pa) )
    
    # --
    g.add( (curr_gara, rel.oggetto, Literal(row['oggetto'])) )
    
    # --
    g.add( (curr_gara, rel.scelta_contraente, Literal(row['sceltaContraente'])) )
    
    # --
    g.add( (curr_gara, rel.importo_aggiudicazione, Literal(float(row['importoAggiudicazione']))) )
    
    # --
    g.add( (curr_gara, rel.importo_somme_liquidate, Literal(float(row['importoSommeLiquidate']))) )
    
    # --
    g.add( (curr_gara, rel.data_inizio, Literal(row['dataInizio'])) )
    
    # --
    g.add( (curr_gara, rel.data_ultimazione, Literal(row['dataUltimazione'])) )
    
    # -- partecipanti
    json_partecipanti = json.loads(row['jsonPartecipanti'])
    if json_partecipanti["partecipante"]:
        for part in json_partecipanti["partecipante"]:
            cf  = part['codiceFiscale']
            ife = part['identificativoFiscaleEstero']
            rs  = part['ragioneSociale']
            
            forni = g.value(None, rel.cf, Literal(cf))
            if not forni:
                forni = URIRef(fornitore + "f" + str(num_forni))
                num_forni += 1
                g.add( (forni, rel.cf, Literal(cf)) )
                g.add( (forni, rel.identificativo_fiscale_estero, Literal(ife)) )
                g.add( (forni, rel.ragione_sociale, Literal(rs)) ) 
                g.add( (forni, RDF.type, tds.fornitore) )
            g.add( (forni, rel.partecipa, curr_gara) )
            
    if json_partecipanti["raggruppamento"]:
        
        for gruppi in json_partecipanti["raggruppamento"]:
            gruppo = gruppi["membro"]
            blank_node = BNode()
            for membro in gruppo:
                cf  = membro['codiceFiscale']
                ife = membro['identificativoFiscaleEstero']
                rs  = membro['ragioneSociale']
                ruolo  = membro['ruolo']

                forni = g.value(None, rel.cf, Literal(cf))
                if not forni:
                    forni = URIRef(fornitore + "f" + str(num_forni))
                    num_forni += 1
                    g.add( (forni, rel.cf, Literal(cf)) )
                    g.add( (forni, rel.identificativo_fiscale_estero, Literal(ife)) )
                    g.add( (forni, rel.ragione_sociale, Literal(rs)) )
                    g.add( (forni, RDF.type, tds.fornitore) ) 
                
                g.add( (forni, URIRef(rel + ruolo[3:]), blank_node) ) 
                g.add( (forni, rel.membro, blank_node) )
                g.add( (blank_node, rel.partecipa, curr_gara) )

    # -- aggiudicatari
    json_aggiudicatari = json.loads(row['jsonAggiudicatari'])
    if json_aggiudicatari["aggiudicatario"]:
        cf  = json_aggiudicatari['aggiudicatario'][0]['codiceFiscale']
        ife = json_aggiudicatari['aggiudicatario'][0]['identificativoFiscaleEstero']
        rs  = json_aggiudicatari['aggiudicatario'][0]['ragioneSociale']
        
        forni = g.value(None, rel.cf, Literal(cf))
        if not forni:
            forni = URIRef(fornitore + "f" + str(num_forni))
            num_forni += 1
            g.add( (forni, RDF.type, tds.fornitore) )
            g.add( (forni, rel.cf, Literal(cf)) )
            g.add( (forni, rel.identificativo_fiscale_estero, Literal(ife)) )
            g.add( (forni, rel.ragione_sociale, Literal(rs)) )
        g.add( (forni, rel.vince, curr_gara) ) 
            
    if json_aggiudicatari["aggiudicatarioRaggruppamento"]:
        array_aggiudicatari = json_aggiudicatari['aggiudicatarioRaggruppamento'][0]['membro']
        cf_membri = [membro['codiceFiscale'] for membro in array_aggiudicatari]
        query_params = ""
        var_alias = 0
        
        for cf in cf_membri:
            query_params = query_params + "?" + str(var_alias) + " rel:cf \"" + cf + "\" . \n\t"
            query_params = query_params + "?" + str(var_alias) + " rel:membro ?n . \n\t"
            var_alias += 1
        
        q = """SELECT DISTINCT ?n
               WHERE {""" + query_params + """
                     ?n rel:partecipa gara:""" + "g" + str(num_gara) + """ .
            }"""
        qres = g.query(q)
        if len(qres) == 1:
            for res in qres:
                g.add( (res[0], rel.vince, curr_gara) ) 
        
    num_gara += 1

### Output data

In [5]:
outpu_file = "/home/chris/Workspace/app/anacDATA/triple/output.ttl"
serialized = g.serialize(format='turtle')

with open(outpu_file, "w+") as file:
    file.write(str(serialized, "utf-8").replace("\\n", "\n"))

In [14]:
s = graph.serialize(format='turtle')
print(str(s).replace("\\n", "\n"))

b'@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix fornitore: <http://td.com/fornitore/> .
@prefix gara: <http://td.com/gara/> .
@prefix pa: <http://td.com/pa/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rel: <http://td.com/schema/rel/> .
@prefix tds: <http://td.com/schema/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

fornitore:foggia rel:vince gara:c .

pa:foggia a foaf:Person ;
    rel:vince gara:c .

tds:pino a foaf:Persona ;
    rel:cf "gigigig" .

'
